In [138]:
import pandas as pd


original = pd.read_csv('data.csv')
original = original.drop(['都道府県名ローマ字', '市区町村名ローマ字', '大字町丁目コード', '大字町丁目名', '緯度', '経度'], axis=1)

prefecture = original.iloc[:,:2]

city = original.iloc[:,2:]
counter = city.iloc[0,1]

for i in range(1, len(city) - 1):
    if city.iloc[i, 1] == counter:
        city = city.drop(index=i)
    else:
        counter = city.iloc[i, 0]

print(city)

KeyboardInterrupt: 